# Approche avec BERT

In [43]:
import warnings
#warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import clean_text

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupShuffleSplit

In [2]:
%time
df_declaration_mrv = pd.read_csv("data/data_mrv/declaration_mrv_complet.csv")#delimiter=';',encoding='ISO-8859-1')
#id_to_dco = pd.read_csv("data/ref_MRV/referentiel_dispositif.csv",delimiter=';',encoding='ISO-8859-1')
#df_effets = pd.read_csv("data/ref_MRV/referentiel_dispositif_effets_connus.csv",delimiter=';',encoding='ISO-8859-1')
#df_dys = pd.read_csv("data/ref_MRV/referentiel_dispositif_dysfonctionnement.csv",delimiter=';',encoding='ISO-8859-1')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.63 µs


In [34]:
%%time
#On complète les effets vide comme étant sans effets
df_declaration_mrv['TYPE_EFFET']  = df_declaration_mrv['TYPE_EFFET'].fillna("PAS D'EFFET NEFASTE DECLARE")
df_declaration_mrv['TEF_ID']= df_declaration_mrv['TEF_ID'].fillna('E1213')

#on selectionne les colonnes avec des effets
df = df_declaration_mrv[['DESCRIPTION_INCIDENT','TYPE_VIGILANCE','LIBELLE_COMMERCIAL',
                         'REFERENCE_COMMERCIALE','ETAT_PATIENT','FABRICANT','DCO_ID',
                         'ACTION_PATIENT','CLASSIFICATION','TEF_ID']]#[df_declaration_mrv['TEF_ID']!='E1213']
# On complète les NaN avec du vide
df['ETAT_PATIENT'] = df['ETAT_PATIENT'].fillna(" ")
df['DESCRIPTION_INCIDENT'] = df['DESCRIPTION_INCIDENT'].fillna(" ")
df['LIBELLE_COMMERCIAL'] = df['LIBELLE_COMMERCIAL'].fillna(" ")
df['FABRICANT'] = df['FABRICANT'].fillna(" ")
df["REFERENCE_COMMERCIALE"] = df['REFERENCE_COMMERCIALE'].fillna(" ")
df['TYPE_VIGILANCE'] = df['TYPE_VIGILANCE'].fillna(" ")
df['CLASSIFICATION'] = df['CLASSIFICATION'].fillna(' ')
df['DCO_ID'] = df['DCO_ID'].fillna(-1)
#On nettoieles variables textueelles : 

for col in  ['DESCRIPTION_INCIDENT','LIBELLE_COMMERCIAL','ETAT_PATIENT','FABRICANT','ACTION_PATIENT'] :
    df[col] = df[col].map(lambda x: clean_text.preprocess_text(x))


df['text'] = df['DESCRIPTION_INCIDENT']+'. '+df['LIBELLE_COMMERCIAL']+'. '+df['ETAT_PATIENT'] +'. '+df['FABRICANT']+'. '+df['ACTION_PATIENT']

features = 'text'
y_col = 'TEF_ID'

print(len(df))
n = 15
# On filtre pour a voir plus de n observations par classse
df = df.groupby("TEF_ID").filter(lambda x: len(x) > n)
print(len(df))
le_v = LabelEncoder()
df.TYPE_VIGILANCE = le_v.fit_transform(df.TYPE_VIGILANCE.values)
le = LabelEncoder()
df.TEF_ID = le.fit_transform(df.TEF_ID.values)

train_index,test_index = next(GroupShuffleSplit(random_state=1029).split(df, groups=df['DESCRIPTION_INCIDENT']))
df_train, df_test = df.iloc[train_index], df.iloc[test_index]
y = df_train.TEF_ID
y_test =df_test.TEF_ID

# Construction de la liste de str
train_list = df_train[features].values.tolist()
test_list =  df_test[features].values.tolist()

76954
76402
CPU times: user 22 s, sys: 168 ms, total: 22.2 s
Wall time: 22.1 s


In [249]:
np.save('y_test_mlb.npy',y_test_lb)

In [277]:
df_m = df.groupby('DESCRIPTION_INCIDENT')['TEF_ID'].apply(list).reset_index(name='multilabels')


df_ = pd.merge(df,df_m, on = 'DESCRIPTION_INCIDENT')
df_['multilabels'] = df_['multilabels'].apply(np.array)
df_['multilabels'] = df_['multilabels'].map(np.unique)

train_index,test_index = next(GroupShuffleSplit(random_state=1029).split(df_, groups=df['DESCRIPTION_INCIDENT']))
df_train, df_test = df_.iloc[train_index], df_.iloc[test_index]
y = df_train.TEF_ID
y_test =df_test.TEF_ID

In [299]:
y_test.shape

(15468, 173)

In [216]:
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
y = df_.multilabels.iloc[train_index]
y_test =df_.multilabels.iloc[test_index]
lb = MultiLabelBinarizer()
y_lb =lb.fit_transform(y) 
y_test_lb = lb.transform(y_test)

In [10]:
print( len(train_list))
print( len(test_list))

61651
14751


In [309]:
%%time
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,f1_score


from sklearn.multiclass import OneVsRestClassifier

file_train = np.load('dgs_camembert_train.npz')
X_train = file_train['arr_0'][:,768:]
file_test =np.load('dgs_camembert_test.npz')
X_test = file_test['arr_0'][:,768:]
y_train = np.load('y_train_mlb.npy')
y_test = np.load('y_test_mlb.npy')

CPU times: user 2.42 s, sys: 432 ms, total: 2.86 s
Wall time: 2.85 s


In [310]:
X_train.shape

(60934, 768)

In [291]:
#y_test = lb.inverse_transform(y_test)
#y_train = lb.inverse_transform(y_train)

In [296]:
LinearSVC(class_weight='balanced')

array([(73,), (73,), (73,), ..., (73,), (73,), (73,)], dtype=object)

In [311]:
%%time
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=200)
X_train_svd = svd.fit_transform(X_train)
X_test_svd = svd.transform(X_test)

CPU times: user 23.7 s, sys: 20.8 s, total: 44.5 s
Wall time: 3.15 s


In [312]:
%%time
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
], )

pipeline.fit(X_train_svd,y_train)
y_pred = pipeline.predict(X_test_svd)

print(f1_score(y_test,y_pred,average='samples'))

0.4286647227351375
CPU times: user 31min 43s, sys: 18.4 s, total: 32min 1s
Wall time: 31min 32s


In [256]:
%%time
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('clf', LinearSVC(class_weight='balanced')),
])
y = df_train.TEF_ID
y_test =df_test.TEF_ID

pipeline.fit(X_train,y)
y_pred = pipeline.predict(X_test)

print(f1_score(y_test,y_pred,average='weighted'))

0.2797106654425564
CPU times: user 8min 11s, sys: 14.9 s, total: 8min 26s
Wall time: 8min 25s


In [ ]:
## Avec un svsd par colonnes

In [313]:
file_train = np.load('results/dgs_camembert_train.npz')
X_train = file_train['arr_0']
file_test =np.load('results/dgs_camembert_test.npz')
X_test = file_test['arr_0']
y_train = np.load('results/y_train_mlb.npy')
y_test = np.load('results/y_test_mlb.npy')

In [314]:
%%time
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
], )

pipeline.fit(X_train_svd,y_train)
y_pred = pipeline.predict(X_test_svd)

print(f1_score(y_test,y_pred,average='samples'))

KeyboardInterrupt: 

In [ ]:
print(accuracy(y_test,y_pred, average = "samples"))

In [271]:
from sklearn.base import BaseEstimator, TransformerMixin

class CamenBertVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model
        self.dim = model.get_word_embedding_dimension()

    def fit(self, X,y=None):
        return self

    def transform(self, X):
        
        return np.array([self.model.encode(sentences,show_progress_bar=True, batch_size=10) for sentences in X])

In [259]:
enc = CamenBertVectorTransformer(model = model)


test = enc.transform(train_list[:10])

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.29s/it]


In [272]:
from sklearn.compose import ColumnTransformer

preprocess = ColumnTransformer(
    [('etat_pat_bert', CamenBertVectorTransformer(model = model), 'ETAT_PATIENT'),
     
     ('description_bert',CamenBertVectorTransformer(model = model), 'DESCRIPTION_INCIDENT'),
     
     ('action_pat_bert',CamenBertVectorTransformer(model = model), 'ACTION_PATIENT'),
     
     ('fabricant_bert',CamenBertVectorTransformer(model = model), 'FABRICANT'),
     
          
    ],
    #
    remainder='passthrough')

X = df_train[:10][['DESCRIPTION_INCIDENT','ETAT_PATIENT','FABRICANT','ACTION_PATIENT']]  
X_prep = preprocess.fit_transform(X)


TypeError: fit() missing 1 required positional argument: 'y'

In [181]:
from sentence_transformers import SentenceTransformer
from numpy import savez_compressed

model = SentenceTransformer('distiluse-base-multilingual-cased')

X_train = model.encode(train_list,show_progress_bar=True, batch_size=10)
X_test = model.encode(test_list,show_progress_bar=True, batch_size=10)

savez_compressed('train.npz', X_train)
savez_compressed('test.npz', X_test)

Batches: 100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


In [182]:
X = np.load('train.npz')

In [179]:
np.squeeze(X).shape

(1000, 512)

In [238]:
model = models.Transformer('camembert-base')

In [245]:
word_embedding_model =  models.Transformer('./Models/Models/speed_camembert_max_len_128_fp16_dynamic_padding_smart_batching_batch_32_seed_321/')

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [246]:
X = model.encode(train_list[:10],show_progress_bar=True, batch_size=10)

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


In [247]:
np.squeeze(X).shape

(10, 1536)

In [193]:
# Use BERT for mapping tokens to embeddings
from sentence_transformers import models
word_embedding_model = models.Transformer('camembert-base')

In [196]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

X = model.encode(train_list[:10],show_progress_bar=True, batch_size=10)

Batches:   0%|          | 0/1 [00:06<?, ?it/s]


KeyError: 'sentence_embedding'

In [195]:
np.squeeze(X).shape

(10, 768)

In [ ]:
# Use BERT for mapping tokens to embeddings
word_embedding_model = models.Transformer('flaubert/flaubert_base_uncased')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [207]:
%%time
import numpy as np
from transformers import  pipeline,FlaubertTokenizer,FlaubertModel, PretrainedConfig, AutoTokenizer, AutoModelWithLMHead

train_batch = np.array_split(train_list[:20], 3)
test_batch = np.array_split(train_list, 1500)

tf_flaubert = 'jplu/tf-flaubert-base-uncased'
flaubert = 'flaubert/flaubert_base_uncased'
camenbert = 'camembert-base'

config = {
    'max_seq_len':68,
    'max_length':68,
    'padding':'max_length',
    'pad_to_max_length' : True,
    'dynamic_padding':False
    }


#model = CamenbertModel.from_pretrained(camenbert,max_length=68)

#tokenizer =CamenbertTokenizer.from_pretrained(camenbert, do_lower_case=True, add_special_tokens=True,
                                             #max_length=68,max_seq_len=68,dynamic_padding =False,
                                             #padding = 'max_length', pad_to_max_length=True)


        
tokenizer = AutoTokenizer.from_pretrained("camembert-base", max_length=68,max_seq_len=68,dynamic_padding =False,
                                        padding = 'max_length', pad_to_max_length=True)

model = AutoModelWithLMHead.from_pretrained("camembert-base")



pipe = pipeline('feature-extraction', model=model, tokenizer=tokenizer)


def compute_feature(batch):
    L = []
    for i  in range(len(batch)) :
        features = pipe(text = batch[i].tolist())
        features = np.squeeze(features)
        print(features.shape)
        L.append(features[:,0,:])
    X = np.concatenate(L)
    return(X)

X = compute_feature(train_batch)

(7, 141, 32005)
(7, 131, 32005)
(6, 196, 32005)
CPU times: user 3min, sys: 11.2 s, total: 3min 11s
Wall time: 1min 10s


In [208]:
from transformers import CamembertModel, CamembertTokenizer

# You can replace "camembert-base" with any other model from the table, e.g. "camembert/camembert-large".
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
camembert = CamembertModel.from_pretrained("camembert-base")

camembert.eval()

CamembertModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, in

In [211]:
%%time
encoded_sentence = tokenizer.encode("J'aime le camembert !")
encoded_sentence = torch.tensor(encoded_sentence).unsqueeze(0)
embeddings, _ = camembert(encoded_sentence)

CPU times: user 15.9 s, sys: 380 ms, total: 16.3 s
Wall time: 2.85 s


In [212]:
from transformers import CamembertConfig
config = CamembertConfig.from_pretrained("camembert-base", output_hidden_states=True)
camembert = CamembertModel.from_pretrained("camembert-base", config=config)

embeddings, _, all_layer_embeddings = camembert(encoded_sentence)
all_layer_embeddings[5]

tensor([[[-0.0032,  0.0075,  0.0040,  ..., -0.0025, -0.0178, -0.0210],
         [-0.0996, -0.1474,  0.1057,  ..., -0.0278,  0.1690, -0.2982],
         [ 0.0557, -0.0588,  0.0547,  ..., -0.0726, -0.0867,  0.0699],
         ...,
         [ 0.0418, -0.3488,  0.3162,  ...,  0.0520,  0.1807, -0.0045],
         [-0.0892,  0.0023, -0.2470,  ..., -0.1322,  0.0132, -0.0541],
         [ 0.0797,  0.2318, -0.1602,  ...,  0.1342, -0.0933,  0.3202]]],
       grad_fn=<NativeLayerNormBackward>)

In [167]:
import torch
tokenizer =FlaubertTokenizer.from_pretrained(flaubert, do_lower_case=True, add_special_tokens=True,max_seq_len=10, padding=10)
features = torch.tensor(tokenizer.BatchEncoding(train_list[:10])).unsqueeze(0)
output = model(features)


AttributeError: 'FlaubertTokenizer' object has no attribute 'BatchEncoding'

In [ ]:
tokenizer.encode

In [165]:
np.shape(output[0])

torch.Size([1, 37, 768])

In [123]:
model.get_output_embeddings(

TypeError: get_output_embeddings() takes 1 positional argument but 2 were given

In [119]:
len(tokenizer.encode(train_list[1],max_length=10,padding=68))

10

In [48]:
f = np.squeeze(f)

In [49]:
f.shape

(5, 68, 768)

In [50]:
X.extend(f)

In [40]:
train_batch[0].tolist()

['mandrin bloqué dans la sonde et impossibilité de le bouger et de le sortir .. quartet. . .. abbott medical france sas. matériel à dispo au sce économat .',
 'mandrin bloqué dans la sonde et impossibilité de le bouger et de le sortir .. sonde stimulation quartet. . .. abbott medical france sas. matériel à dispo au sce économat .',
 'lors de ouverture un pack cataracte ( personnalisé ) et du couteau ophtalmique , il est découvert des traces de rouilles sur la lame du couteau . pas de conséquences car non utilisation du dm. couteau standart ° droit st. pas de conséquences car non utilisation du dm .. bausch & lomb france. changements de couteaux .',
 'lors de ouverture un pack cataracte ( personnalisé ) et du couteau ophtalmique , il est découvert des traces de rouilles sur la lame du couteau . pas de conséquences car non utilisation du dm. couteau standart ° droit st. pas de conséquences car non utilisation du dm .. bausch & lomb france. changements de couteaux .',
 'lors de ouverture 

In [29]:
train_batch[0]

array(['m', 'm', 'l', 'l', 'l'], dtype='<U1')

In [18]:
Batch = np.array_split(train_list, 3)

In [6]:
train_list = [elt[0] for elt in df_train[features].values.tolist()]
test_list =  [elt[0] for elt in df_test[features].values.tolist()]

In [20]:
print(len(Batch[1]))

20550


# using traformers

In [25]:
import transformers
from transformers import glue_convert_examples_to_features
from transformers import TFBertForSequenceClassification
import tensorflow as tf
from transformers import DistilBertTokenizer, RobertaTokenizer ,FlaubertTokenizer

In [24]:
# construction tokenizer

distil_bert = 'distilbert-base-uncased' # Pick any desired pre-trained model
roberta = 'roberta-base'

# Defining DistilBERT tokonizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)
# Defining RoBERTa tokinizer
tokenizer = RobertaTokenizer.from_pretrained(roberta, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)

In [16]:
df_train.iloc[0].index()

TypeError: 'Index' object is not callable

In [32]:
# construction des Imput
class Example:
    text: str
    label: int


def Example_from_df(df,features,target):
    Sentences = list()
    for ind in df.index :
        line = df.loc[ind]
        text =str()
        ex = Example()
        for feature in features :
            text = text +'. '+ line[feature]
        ex.text = text
        ex.label = float(line[target])
        Sentences.append(ex) 
    return(Sentences)
        
        

In [33]:
train_Example = Example_from_df(df_train,features,y_col)

In [38]:
train_Example[0].label

73.0

In [39]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-cased")

train_dataset = glue_convert_examples_to_features(train_Example, tokenizer, 128)




TypeError: 'NoneType' object is not iterable

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
model.fit(train_dataset, epochs=3)

In [ ]:



# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5
# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1
# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
# classifier Adam recommended
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [12]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

distil_bert = 'flaubert-base-uncased'

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(50, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(6, activation='sigmoid')(X)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

for layer in model.layers[:3]:
    layer.trainable = False

In [27]:
%%time
import numpy as np
from transformers import  pipeline,FlaubertTokenizer,TFFlaubertModel

flaubert = 'jplu/tf-flaubert-base-uncased'

model = TFFlaubertModel.from_pretrained(flaubert,max_length=512)

tokenizer =FlaubertTokenizer.from_pretrained(flaubert, do_lower_case=True, add_special_tokens=True,
                                                max_length=512, pad_to_max_length=True)

pipe = pipeline('feature-extraction', model=model, 
                tokenizer=tokenizer)

features = pipe(train_list[:1000],
                pad_to_max_length=True)


#features = np.squeeze(features)

#X_train = features[:,0,:]

CPU times: user 1h 42min 52s, sys: 32min 36s, total: 2h 15min 28s
Wall time: 16min 38s


In [28]:
features = np.squeeze(features)

X_train = features[:,0,:]

In [31]:
features.shape

(1000, 339, 768)

In [21]:
import numpy as np
from transformers import AutoTokenizer, pipeline, DistilBertModel,DistilBertTokenizer
distil_bert = 'distilbert-base-uncased'
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=512, pad_to_max_length=False)
pipe = pipeline('feature-extraction', model=model,
                tokenizer=tokenizer)

features = pipe(train_list[:1000],
                pad_to_max_length=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (511 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (511 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (511 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (511 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (511 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

ValueError: The sequences building the batch are not of the same size, no tensor can be built. Set `pad_to_max_length=True` to pad the smaller sequencesup to the larger sequence's length.